In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
orders_df = spark.read \
.format("csv") \
.option("inferSchema","true") \
.option("header","true") \
.load("/public/trendytech/datasets/windowdatamodified.csv")

In [6]:
from pyspark.sql.functions import *

In [8]:
from pyspark.sql import *

In [9]:
mywindow = Window.partitionBy("country") \
.orderBy(desc("invoicevalue"))

In [10]:
result_df = orders_df.withColumn("previous_week",lag("invoicevalue").over(mywindow))

In [11]:
result_df.show()

+-------+-------+-----------+-------------+------------+-------------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|previous_week|
+-------+-------+-----------+-------------+------------+-------------+
| Sweden|     50|          3|         3714|      2646.3|         null|
|Germany|     49|         12|         1852|      1800.0|         null|
|Germany|     50|         15|         1973|      1800.0|       1800.0|
|Germany|     48|         11|         1795|      1600.0|       1800.0|
|Germany|     51|          5|         1103|      1600.0|       1600.0|
| France|     50|          6|          529|      537.32|         null|
| France|     51|          5|          847|       500.0|       537.32|
| France|     49|          9|         2303|       500.0|        500.0|
| France|     48|          4|         1299|       500.0|        500.0|
|Belgium|     48|          1|          528|       800.0|         null|
|Belgium|     51|          2|          942|       800.0|        800.0|
|Belgi

In [14]:
final_df = result_df.withColumn("invoice_difference",expr("invoicevalue - previous_week"))

In [15]:
final_df.show()

+-------+-------+-----------+-------------+------------+-------------+-------------------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|previous_week| invoice_difference|
+-------+-------+-----------+-------------+------------+-------------+-------------------+
| Sweden|     50|          3|         3714|      2646.3|         null|               null|
|Germany|     49|         12|         1852|      1800.0|         null|               null|
|Germany|     50|         15|         1973|      1800.0|       1800.0|                0.0|
|Germany|     48|         11|         1795|      1600.0|       1800.0|             -200.0|
|Germany|     51|          5|         1103|      1600.0|       1600.0|                0.0|
| France|     50|          6|          529|      537.32|         null|               null|
| France|     51|          5|          847|       500.0|       537.32| -37.32000000000005|
| France|     49|          9|         2303|       500.0|        500.0|                0.0|

In [16]:
mywindow = Window.partitionBy("country")

In [19]:
result = orders_df.withColumn("total_invoice_value",sum("invoicevalue").over(mywindow))

In [20]:
result.show()

+-------+-------+-----------+-------------+------------+-------------------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|total_invoice_value|
+-------+-------+-----------+-------------+------------+-------------------+
| Sweden|     50|          3|         3714|      2646.3|             2646.3|
|Germany|     48|         11|         1795|      1600.0|             6800.0|
|Germany|     49|         12|         1852|      1800.0|             6800.0|
|Germany|     50|         15|         1973|      1800.0|             6800.0|
|Germany|     51|          5|         1103|      1600.0|             6800.0|
| France|     51|          5|          847|       500.0| 2037.3200000000002|
| France|     49|          9|         2303|       500.0| 2037.3200000000002|
| France|     48|          4|         1299|       500.0| 2037.3200000000002|
| France|     50|          6|          529|      537.32| 2037.3200000000002|
|Belgium|     48|          1|          528|       800.0|            2225.16|